In [106]:
import numpy as np
import pandas as pd

In [107]:
from sklearn import datasets, svm, metrics, linear_model, cross_validation

# 1. 虚拟变量
写一个函数，能将一个多类别变量转为多个二元虚拟变量，不能使用sklearn库。

In [108]:
def get_dummies(x):
    columns = set(x)
    n_col = len(columns)
    n_row = len(x)
    data = pd.DataFrame(data=np.zeros((n_row, n_col)), columns=columns)
    for i, c in enumerate(x):
        data.ix[i, c] = 1
    return data    

In [109]:
s = pd.Series(list('abca'))

get_dummies(s)

,a,c,b
0,1,0,0
1,0,0,1
2,0,1,0
3,1,0,0


# 2.交叉验证
写一个函数，实现交叉验证的功能，不能使用sklearn库。

In [110]:
def train_test_split(X, y, train_size=0.5):
    index = range(len(y))
    np.random.shuffle(index)
    n_boundary = int(len(index) * train_size)
    train_index = index[: n_boundary]
    test_index = index[n_boundary :]
    return X[train_index, :], y[train_index], X[test_index, :], y[test_index]

In [111]:
X = np.array([np.random.randn(3) for i in range(10)])
y = np.random.randn(10)
train_test_split(X, y)

(array([[-1.39149492,  0.34839788,  1.99201894],
        [-0.63772096,  0.82433084,  0.45521973],
        [ 0.30488434,  0.35193031, -0.00579267],
        [ 2.02876686, -0.01110466,  1.02082122],
        [ 0.5538864 ,  0.05560918,  1.50530195]]),
 array([-1.09280362,  1.08879329, -0.12227818,  0.04920672,  1.09713555]),
 array([[-1.7083035 , -0.87392321, -1.37900819],
        [ 0.2008686 ,  1.84292056,  1.29818021],
        [-0.26734174,  1.66768661,  1.57657683],
        [ 2.33362911, -1.6977674 ,  0.4409638 ],
        [-0.27610998,  1.54173327, -0.41585132]]),
 array([-1.9544055 ,  0.110926  ,  0.23513772,  0.55620588,  1.24390989]))

# 3.分类
使用sklearn库中的其他分类方法，来预测titanic的生存情况。

（需要到 [Kaggle](https://www.kaggle.com/c/titanic) 上下载数据)

In [112]:
train_df = pd.read_csv('data/titanic/train.csv')
train_df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S


检查缺失数据

In [113]:
train_df.isnull().sum(0)

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

用中位数填补 Age 列

In [114]:
train_df.ix[(train_df.Age.isnull()), 'Age'] = train_df['Age'].dropna().median()

编辑 Sex 列

In [115]:
train_df['Sex'] = train_df['Sex'].map({'female': 0, 'male': 1}).astype(int)

去掉不感兴趣的变量

In [116]:
train_df = train_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
train_df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22,1,0,7.2500
1,1,1,0,38,1,0,71.2833
2,1,3,0,26,0,0,7.9250


对测试集数据进行同样操作

In [117]:
test_df = pd.read_csv('data/titanic/test.csv')
test_df.ix[(test_df.Age.isnull()), 'Age'] = test_df['Age'].dropna().median()
test_df.ix[test_df['Fare'].isnull(), 'Fare'] = test_df['Fare'].dropna().median()
test_df['Sex'] = test_df['Sex'].map({'female': 0, 'male': 1}).astype(int)
test_df = test_df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1) 
test_df.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,1,34.5,0,0,7.8292
1,3,0,47.0,1,0,7.0000
2,2,1,62.0,0,0,9.6875


In [118]:
train_data = train_df.values
test_data = test_df.values

用 svm 方法

In [119]:
classifier = svm.SVC(gamma=0.001)
classifier.fit(train_data[:, 1:], train_data[:, 0])

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3,
  gamma=0.001, kernel='rbf', max_iter=-1, probability=False,
  random_state=None, shrinking=True, tol=0.001, verbose=False)

预测结果为

In [120]:
classifier.predict(test_data)

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  1.,  0.,
        1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,
        0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,
        0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
        0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        1.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
        0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
        1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  1.,  0

# 4.特征工程
研究kaggle中的[Digit Recognizer数据](https://www.kaggle.com/c/digit-recognizer/data)，尝试用一些特征工程来提取数字的特征，并放入分类器中观察预测准确率，相对直接使用原始变量是否有提升。

（需要到 Kaggle 上下载数据)

In [121]:
df = pd.read_csv('data/digit/train.csv', nrows=1000)
df.head(3)

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [122]:
train_data, test_data = cross_validation.train_test_split(df.values, train_size=0.8, random_state=100)

In [123]:
classifier = linear_model.LogisticRegression()
classifier.fit(train_data[:, 1:], train_data[:, 0])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [124]:
expected = test_data[:, 0]
predicted = classifier.predict(test_data[:, 1:])

预测结果

In [125]:
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

          0       0.89      0.94      0.92        54
          1       0.84      0.93      0.89        46
          2       0.91      0.82      0.87        74
          3       0.88      0.71      0.78        51
          4       0.94      0.52      0.67        63
          5       0.67      0.67      0.67        49
          6       0.84      0.91      0.87        45
          7       0.69      0.84      0.76        43
          8       0.63      0.69      0.66        39
          9       0.45      0.69      0.54        36

avg / total       0.80      0.77      0.77       500



混淆矩阵

In [126]:
print(metrics.confusion_matrix(expected, predicted))

[[51  0  0  0  0  1  1  0  1  0]
 [ 0 43  0  0  0  0  0  0  2  1]
 [ 2  2 61  0  0  0  2  3  2  2]
 [ 1  1  4 36  0  5  0  1  1  2]
 [ 1  0  0  0 33  0  2  6  5 16]
 [ 0  2  1  4  2 33  3  1  0  3]
 [ 0  0  0  0  0  3 41  0  1  0]
 [ 1  1  0  0  0  0  0 36  1  4]
 [ 0  2  0  0  0  7  0  0 27  3]
 [ 1  0  1  1  0  0  0  5  3 25]]


接下来使用 HOG 特征

In [127]:
from skimage.feature import hog

In [128]:
features = df.values[:, 1:]
label = df.values[:, 0]

In [131]:
list_hog_fd = []
for feature in features:
    fd = hog(feature.reshape((28, 28)), orientations=9, 
             pixels_per_cell=(14, 14), cells_per_block=(1, 1), visualise=False)
    list_hog_fd.append(fd)
hog_features = np.array(list_hog_fd, 'float64')

In [133]:
train_features, test_features, train_label ,test_label = cross_validation.train_test_split(hog_features, label, 
                                                                                           train_size=0.8, 
                                                                                           random_state=100)

In [134]:
classifier = linear_model.LogisticRegression()
classifier.fit(train_features, train_label)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [135]:
expected = test_label
predicted = classifier.predict(test_features)

预测结果

In [136]:
print(metrics.classification_report(expected, predicted))

             precision    recall  f1-score   support

          0       0.81      0.94      0.87        54
          1       0.75      0.87      0.81        46
          2       0.87      0.89      0.88        74
          3       0.89      0.76      0.82        51
          4       0.97      0.48      0.64        63
          5       0.89      0.82      0.85        49
          6       0.81      0.98      0.89        45
          7       0.88      0.86      0.87        43
          8       0.71      0.74      0.72        39
          9       0.43      0.61      0.51        36

avg / total       0.82      0.80      0.79       500



混淆矩阵

In [137]:
print(metrics.confusion_matrix(expected, predicted))

[[51  1  0  0  0  0  0  0  0  2]
 [ 1 40  1  0  0  0  0  1  1  2]
 [ 1  0 66  1  1  0  0  0  3  2]
 [ 2  0  3 39  0  2  0  1  0  4]
 [ 0 10  1  0 30  1  7  1  3 10]
 [ 0  1  0  0  0 40  0  0  4  4]
 [ 1  0  0  0  0  0 44  0  0  0]
 [ 0  0  3  0  0  0  0 37  0  3]
 [ 1  0  1  1  0  2  2  1 29  2]
 [ 6  1  1  3  0  0  1  1  1 22]]


相比使用原始特征，使用 HOG 特征能提升预测准确率